In [1]:
import pandas as pd

data = pd.read_csv("complete_data_minmax.csv")

In [ ]:
hdi = pd.read_csv("HDI/hdr-data.csv")
hdi = hdi[["countryIsoCode","year", "indicatorCode","value" ]]
hdi = hdi[hdi['countryIsoCode'].str.len() <= 3]
hdi = hdi[(hdi['indicatorCode'] == 'hdi')]
hdi = hdi.drop(columns=["indicatorCode"])
hdi.rename(columns={'countryIsoCode': 'SpatialDimValueCode', 'year': 'Period', 'value': 'Hdi'}, inplace=True)
hdi = hdi.set_index(["SpatialDimValueCode", "Period"])
hdi

In [ ]:
merged = pd.merge(hdi, homicide_df, left_index=True, right_index=True)
merged = merged.merge(he_df, on= ["SpatialDimValueCode", "Period"]).drop(columns=["childHomeVisits","elderCare","partnerVioPrevention", "youthPrograms"])


In [68]:
merged

,SpatialDimValueCode,Period,Hdi,HomicideRate,FactValueNumeric,Health_Expenditure
0,AFG,2000,0.340,9.78,7.71,NaN
1,AFG,2001,0.344,9.64,7.89,NaN
2,AFG,2002,0.368,10.18,7.85,9.0
3,AFG,2003,0.379,10.40,7.72,9.0
4,AFG,2004,0.395,9.74,7.77,10.0
...,...,...,...,...,...,...
3551,ZWE,2015,0.544,13.43,30.74,7.0
3552,ZWE,2016,0.547,13.51,28.70,7.0
3553,ZWE,2017,0.552,13.28,25.85,6.0
3554,ZWE,2018,0.564,13.19,23.87,5.0


In [99]:
def map_values(value):
    if isinstance(value, str):
        return value

    if value > 1000:
        return value
    return int(value * 10) + 1 if value < 1 else 10


classes_pd = pd.read_csv("complete_data_minmax_test.csv").drop("Unnamed: 0", axis=1).map(map_values)

In [106]:
def convert_rows_to_strings(df):
    result = []
    for index, row in df.iterrows():
        row_str = [f"{col}:{row[col]}" for col in df.columns]
        result.append(row_str)
    return result

convert_rows_to_strings(classes_pd)

[['SpatialDimValueCode:AFG',
  'HealthAvailVal_nurse:1',
  'HealthAvailVal_psych:1',
  'Period:2019',
  'FactValueNumeric_alcohol:1',
  'suicide_rate:1',
  'HDI:4',
  'Health_Expenditure:8',
  'HomicideRate:1'],
 ['SpatialDimValueCode:AFG',
  'HealthAvailVal_nurse:1',
  'HealthAvailVal_psych:1',
  'Period:2018',
  'FactValueNumeric_alcohol:1',
  'suicide_rate:1',
  'HDI:4',
  'Health_Expenditure:7',
  'HomicideRate:1'],
 ['SpatialDimValueCode:AFG',
  'HealthAvailVal_nurse:1',
  'HealthAvailVal_psych:1',
  'Period:2017',
  'FactValueNumeric_alcohol:1',
  'suicide_rate:1',
  'HDI:4',
  'Health_Expenditure:7',
  'HomicideRate:1'],
 ['SpatialDimValueCode:AFG',
  'HealthAvailVal_nurse:1',
  'HealthAvailVal_psych:1',
  'Period:2016',
  'FactValueNumeric_alcohol:1',
  'suicide_rate:1',
  'HDI:4',
  'Health_Expenditure:6',
  'HomicideRate:1'],
 ['SpatialDimValueCode:AFG',
  'HealthAvailVal_nurse:1',
  'HealthAvailVal_psych:1',
  'Period:2015',
  'FactValueNumeric_alcohol:1',
  'suicide_rate:1'

In [ ]:
import pyfpgrowth
# Convert the Query column to a list of lists
associations = convert_rows_to_strings(classes_pd)
num_records = len(associations)

sigma = 100
min_support = sigma / num_records
print(f"Minimum support: {min_support}")
# the function find_frequent_patterns takes SIGMA as second parameter
patterns = pyfpgrowth.find_frequent_patterns(associations, sigma)
patterns

In [208]:
for k,v in patterns.items():
    if "suicide_rate:4" in str(k):
        print(k,v)

('suicide_rate:4',) 145
('HealthAvailVal_nurse:1', 'suicide_rate:4') 103


In [129]:
confidence = 0.5
rules = pyfpgrowth.generate_association_rules(patterns, confidence)
implySui = []
suiImply = []
for k,v in rules.items():
    if "suicide_rate" in str(v):
        implySui.append({k:rules[k]})
        print(f"{k} -> {v}")
    if "suicide_rate" in str(k):
        suiImply.append({k:rules[k]})

('FactValueNumeric_alcohol:6', 'HomicideRate:1') -> (('suicide_rate:2',), 0.5511111111111111)
('HDI:10', 'HomicideRate:1') -> (('suicide_rate:2',), 0.7631578947368421)
('FactValueNumeric_alcohol:1', 'HDI:6', 'HealthAvailVal_nurse:1', 'HealthAvailVal_psych:1') -> (('suicide_rate:1',), 0.8907563025210085)
('FactValueNumeric_alcohol:3', 'HealthAvailVal_nurse:1', 'HealthAvailVal_psych:1') -> (('suicide_rate:1',), 0.536)
('FactValueNumeric_alcohol:2', 'HealthAvailVal_nurse:1', 'HealthAvailVal_psych:1', 'HomicideRate:1') -> (('suicide_rate:2',), 0.645)
('HDI:8', 'HealthAvailVal_nurse:1', 'HealthAvailVal_psych:1', 'HomicideRate:1') -> (('suicide_rate:1',), 0.7926829268292683)
('FactValueNumeric_alcohol:1', 'HDI:7', 'HealthAvailVal_nurse:1') -> (('HealthAvailVal_psych:1', 'HomicideRate:1', 'suicide_rate:1'), 0.9107142857142857)
('FactValueNumeric_alcohol:1', 'HDI:7', 'HealthAvailVal_nurse:1', 'HealthAvailVal_psych:1') -> (('HomicideRate:1', 'suicide_rate:1'), 0.9107142857142857)
('FactValueNum

In [167]:
implySui

[{('FactValueNumeric_alcohol:6', 'HomicideRate:1'): (('suicide_rate:2',),
   0.5511111111111111)},
 {('HDI:10', 'HomicideRate:1'): (('suicide_rate:2',), 0.7631578947368421)},
 {('FactValueNumeric_alcohol:1',
   'HDI:6',
   'HealthAvailVal_nurse:1',
   'HealthAvailVal_psych:1'): (('suicide_rate:1',), 0.8907563025210085)},
 {('FactValueNumeric_alcohol:3',
   'HealthAvailVal_nurse:1',
   'HealthAvailVal_psych:1'): (('suicide_rate:1',), 0.536)},
 {('FactValueNumeric_alcohol:2',
   'HealthAvailVal_nurse:1',
   'HealthAvailVal_psych:1',
   'HomicideRate:1'): (('suicide_rate:2',), 0.645)},
 {('HDI:8',
   'HealthAvailVal_nurse:1',
   'HealthAvailVal_psych:1',
   'HomicideRate:1'): (('suicide_rate:1',), 0.7926829268292683)},
 {('FactValueNumeric_alcohol:1',
   'HDI:7',
   'HealthAvailVal_nurse:1'): (('HealthAvailVal_psych:1',
    'HomicideRate:1',
    'suicide_rate:1'), 0.9107142857142857)},
 {('FactValueNumeric_alcohol:1',
   'HDI:7',
   'HealthAvailVal_nurse:1',
   'HealthAvailVal_psych:1'): 

In [141]:
import graphviz
dot = graphviz.Digraph(comment='Association Rules')
dot.node("hej0", "hek2")
dot.render('association_rules_graph', format='png', view=True)
list(implySui[0].items())[0][0]

('FactValueNumeric_alcohol:6', 'HomicideRate:1')

/snap/core20/current/lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.29' not found (required by /lib/x86_64-linux-gnu/libproxy.so.1)
Failed to load module: /home/nils/snap/code/common/.cache/gio-modules/libgiolibproxy.so


In [194]:
dot = graphviz.Digraph(comment='Association Rules')
dot.attr(rankdir='LR')

knownNodes =  []
for ass in implySui:
    # print(ass)
    print(list(ass.values())[0][1])
    keys_str = ''.join(str(k).replace("'", "") for k in ass.keys()).replace("(", "").replace(")", "").replace(":",";")
    values_str = ''.join(str(v[0]).replace("'", "") for v in ass.values()).replace("(", "").replace(")", "").replace(":",";")
    # print(f"Keys: {keys_str}")
    # print(f"Values: {values_str}")
    
    dot.edge(keys_str,values_str, str(list(ass.values())[0][1]))
    


        
dot.render('association_rules_graph', format='png', view=True)
    

0.5511111111111111
0.7631578947368421
0.8907563025210085
0.536
0.645
0.7926829268292683
0.9107142857142857
0.9107142857142857
0.9107142857142857
0.9285714285714286
0.9107142857142857
0.5675675675675675
0.6265060240963856
0.5658914728682171
0.5235602094240838
0.6029629629629629


'association_rules_graph.png'

/snap/core20/current/lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.29' not found (required by /lib/x86_64-linux-gnu/libproxy.so.1)
Failed to load module: /home/nils/snap/code/common/.cache/gio-modules/libgiolibproxy.so


In [132]:
import graphviz
# Sample rules extracted from the provided data
rulesSample = {
    ('FactValueNumeric_alcohol:6', 'HomicideRate:1'): (('suicide_rate:2',), 0.5511111111111111),
    ('HDI:10', 'HomicideRate:1'): (('suicide_rate:2',), 0.7631578947368421),
    ('FactValueNumeric_alcohol:1', 'HDI:6', 'HealthAvailVal_nurse:1', 'HealthAvailVal_psych:1'): (('suicide_rate:1',), 0.8907563025210085),
    # Add more rules as needed
}

# Initialize a Graphviz Digraph
dot = graphviz.Digraph(comment='Association Rules')

# Add edges to the graph
for cause, (effect, confidence) in rulesSample.items():
    cause_str = ', '.join(cause)
    effect_str = ', '.join(effect)
    dot.edge(cause_str, effect_str, label=f'confidence: {confidence:.2f}')

# Render the graph to a file or display it
dot.render('association_rules_graph', format='png', view=True)

'association_rules_graph.png'

/snap/core20/current/lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.29' not found (required by /lib/x86_64-linux-gnu/libproxy.so.1)
Failed to load module: /home/nils/snap/code/common/.cache/gio-modules/libgiolibproxy.so


In [133]:
import graphviz

# Sample rules extracted from the provided data
rulesSample = {
    ('FactValueNumeric_alcohol:6', 'HomicideRate:1'): (('suicide_rate:2',), 0.5511111111111111),
    ('HDI:10', 'HomicideRate:1'): (('suicide_rate:2',), 0.7631578947368421),
    ('FactValueNumeric_alcohol:1', 'HDI:6', 'HealthAvailVal_nurse:1', 'HealthAvailVal_psych:1'): (('suicide_rate:1',), 0.8907563025210085),
    # Add more rules as needed
}

# Initialize a Graphviz Digraph
dot = graphviz.Digraph(comment='Association Rules')

# Add edges to the graph
for cause, (effect, confidence) in rulesSample.items():
    cause_str = ', '.join(cause)
    for eff in effect:
        dot.node(eff, eff)
        for c in cause:
            dot.node(c, c)
            dot.edge(c, eff, label=f'confidence: {confidence:.2f}')

# Render the graph to a file or display it
dot.render('association_rules_graph', format='png', view=True)

'association_rules_graph.png'

/snap/core20/current/lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.29' not found (required by /lib/x86_64-linux-gnu/libproxy.so.1)
Failed to load module: /home/nils/snap/code/common/.cache/gio-modules/libgiolibproxy.so
